In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

file_path = 'AllNullsFilled.csv'
data = pd.read_csv(file_path, low_memory=False)

balloons_of_interest = [
    "Eclipse Pod v2 - IMS Pre",
    "Eclipse Pod v2 - IMS During",
    "Eclipse Pod v2 - GRCC",
    "Eclipse Pod v2 - PFW Edu"
]

data['PercentTotality'] = pd.to_numeric(data['PercentTotality'], errors='coerce')
filtered_data = data.dropna(subset=['PercentTotality'])

filtered_data = filtered_data[
    (filtered_data['Balloon'].isin(balloons_of_interest)) &
    (filtered_data['AltImputed'] >= 0) &
    (filtered_data['AltImputed'] <= 30000)
]

filtered_data['Timestamp'] = pd.to_datetime(filtered_data['Timestamp']).dt.tz_localize(None)

time_ranges = {
    "Eclipse Pod v2 - IMS Pre": ('2024-04-08 12:00:00', '2024-04-08 13:45:00'),
    "Eclipse Pod v2 - IMS During": ('2024-04-08 14:00:00', '2024-04-08 16:00:00'),
    "Eclipse Pod v2 - GRCC": ('2024-04-08 14:00:00', '2024-04-08 16:00:00'),
    "Eclipse Pod v2 - PFW Edu": ('2024-04-08 14:30:00', '2024-04-08 17:00:00')
}

fig, axes = plt.subplots(2, 2, figsize=(18, 12), sharex=False, sharey=True)

norm = plt.Normalize(0, 1)

subplot_labels = ['A', 'B', 'C', 'D']

for ax, balloon, label in zip(axes.flatten(), balloons_of_interest, subplot_labels):
    balloon_data = filtered_data[filtered_data['Balloon'] == balloon]
    start_time, end_time = time_ranges[balloon]
    balloon_data = balloon_data[
        (balloon_data['Timestamp'] >= pd.Timestamp(start_time)) &
        (balloon_data['Timestamp'] <= pd.Timestamp(end_time))
    ]
    if balloon_data.empty:
        ax.text(0.5, 0.5, 'No data available', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    else:
        sc = ax.scatter(balloon_data['Timestamp'], balloon_data['Temperature (F)'], c=balloon_data['PercentTotality'], cmap='viridis_r', alpha=0.7, norm=norm)
        ax.set_title(f'{label}: {balloon}')
        ax.set_xlabel('Timestamp')
        ax.set_ylabel('Temperature (F)')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        cbar = fig.colorbar(sc, ax=ax, orientation='vertical')
        cbar.set_label('PercentTotality')
    ax.set_xlim(pd.Timestamp(start_time), pd.Timestamp(end_time))
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_ha('right')

plt.tight_layout()
plt.show()


KeyError: 'PercentTotality'